## Only topic model - inv - exp

In [ ]:
from model import *

In [ ]:
from types import SimpleNamespace
parser = {}
parser['gpu'] = '0'
parser['seed']=int(time.time())#1234
parser['log_period']=200#625#3000
parser['max_to_keep']=10
parser['n_freq'] = 10
parser['n_epochs']=410#1000
parser['batch_size']=512
parser['opt'] = 'Adagrad'
parser['lr']=0.01#0.005
parser['reg']=0.1
parser['reg3']=1000.
parser['reg4']=1.

parser['keep_prob']=0.8
parser['dim_hidden_bow']=100#256
parser['dim_latent_bow']=2#2#100
parser['dim_latent_path']=2#100

parser['dim_emb']=256#256
parser['tree']=63#121#126#63#126
parser['n_depth']=4
parser['depth_temperature']=0.1#10
#parser['add_threshold']={1: 0.065, 2:0.04} #> add

# min_child = 3 # add_threshold (1/min_child) ** level, max_level(r) > add_threshold
# max_child = 5
# max_depth = 3

#parser['remove_threshold']=0.01 #< remove, #(1/max_child) ** level
parser['cell']='rnn'
parser['static']=False
# parser['path_data']='data/jacm/instances.pkl'
# parser['dir_model']='model/jacm/checkpoint_tmp'

parser['data_type'] ='bbc'
parser['dist_type']='inv'
parser['path_data']='data/'+parser['data_type']+'/instances.pkl'
# parser['path_data']='data/20news/instances.pkl'
parser['dir_model']='model/'+parser['data_type']+'/checkpoint_tmp'
# parser['train_labels']='data/20news_with_labels/train_20news_labels.npy'
parser['tmp']=False

doc_len = {'bbc': 2225, 'reuters':7674, '20news':18181, 'mendeley': 46985}
parser['tree_expected_min_child'] = 5# math.ceil(math.ceil(np.cbrt(doc_len[parser['data_type']]/2.))/3.)#3#6#3#6
parser['tree_expected_max_child'] = 10#math.ceil(np.cbrt(doc_len[parser['data_type']]/2.))#6#15#6#15
parser['tree_max_depth'] = 4

parser['vis']=True

config = SimpleNamespace(**parser)

In [3]:
for n_time in range(1):
    n_time=3
    config.vis=True
    config.tree_idxs = get_tree_idxs(config.tree)
    config.path_model = os.path.join(config.dir_model, 'model') 
    config.path_config = config.path_model + '-%i.config'
    config.path_log = os.path.join(config.dir_model, 'log')
    config.path_checkpoint = os.path.join(config.dir_model, 'checkpoint')

    os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
    np.random.seed(config.seed)
    random.seed(config.seed)
    # load data
    instances_train, _, _, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
    train_batches = get_batches(instances_train, config.batch_size)
    #dev_batches = get_batches(instances_dev, config.batch_size)
    config.dim_bow = len(bow_idxs)

    # initialize log
    checkpoint = []
    losses_train = []
    ppls_train = []
    ppl_min = np.inf
    epoch = 0
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

    log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                        list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','', 'SPEC:', '', '', 'HIER:', ''],
                                ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG', '1', '2', '3', 'CHILD', 'OTHER']]))))


    # initialize model
    if 'sess' in globals(): sess.close()
    model = HierarchicalNeuralTopicModel(config)
    # sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
    sess = tf.InteractiveSession(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
    sess.run(tf.global_variables_initializer())
    #========================== FR ====================
    v = model.topic_idxs
    topic_coords = sess.run(model.topic_coords)
    tree = model.tree_idxs
    pairs = [[(parent,j) for j in childs] for parent,childs in tree.items()]
    flat_pairs = [item for sublist in pairs for item in sublist]
    G = nx.Graph()
    G.add_nodes_from(v)
    G.add_edges_from(flat_pairs)
    # initial position
    for v in nx.nodes(G):
        G.nodes[v]['x'] = topic_coords[v][0][0]
        G.nodes[v]['y'] = topic_coords[v][0][1]
    pos = {}
    for v in G.nodes():
        pos[v] = [G.nodes[v]['x'],G.nodes[v]['y']]

#     pos=nx.fruchterman_reingold_layout(G, scale=0.1)    
    pos=nx.kamada_kawai_layout(G, scale=0.1)    

    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
    for name in name_tensors.keys():
        if ('Adagrad' not in name) and ('topic_coords' in name):
            node_idx = int(name.split('topic_coords')[1].split(':')[0])
            new_posx = pos[node_idx][0]
            new_posy = pos[node_idx][1]

            new_arr = np.array([[new_posx, new_posy]], dtype=np.float32)
            sess.run([name_tensors[name].assign(new_arr)])

    #==========================

    saver = tf.train.Saver(max_to_keep=config.max_to_keep)
    update_tree_flg = False

    # train model
    time_start = time.time()
    time_start = time.time()
    print('---START TRAINING----------')
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    dev_batches = get_batches(instances_train, config.batch_size)
    while epoch < config.n_epochs:
        losses_train = []
        ppls_train = []
        for ct, batch in enumerate(train_batches):
            feed_dict, current_batch_size = model.get_feed_dict(batch, epoch)

            if current_batch_size!=config.batch_size:
                continue

            _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, \
                                                    tree_prob_leaf, prob_depth, topic_bow, gamma_topic, topic_coords, reg3, prob_topic,\
                                                    grad_vars, logvars_bow,\
                                                                                                            global_step_log = \
            sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, \
                                           model.tree_prob_leaf, model.prob_depth, model.topic_bow, model.gamma_topic, model.topic_coords,model.coord_reg3, model.prob_topic,\
                                            model.grad_vars, model.logvars_bow,\
                                                                                                            tf.train.get_global_step()], feed_dict = feed_dict)


            losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, reg3]]

            ppls_train += list(ppls_batch)

        if (epoch % config.log_period==0) and (epoch>0) and (epoch!=config.n_epochs):#global_step_log % config.log_period == 0:#config.log_period
            print('epoch: ', epoch)
        # if ct%10==0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, reg3 = np.mean(losses_train, 0)#losses_train#np.mean(losses_train, 0)
            # print(loss_train)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, epoch, model, config)

            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

            depth_specs = compute_topic_specialization(sess, model, instances_train)
            hierarchical_affinities = compute_hierarchical_affinity(sess, model)

            # save log
            #### PRINT
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            print(log_series)
            #### PRINT
            #cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                if epoch>=2000:
                    disable_add = True
                else:
                    disable_add = False
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic, disable_add)
                if update_tree_flg:

                    prev_name_tensors = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()

                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    #=== INIT NON INIT VARS=======
                    non_init_coords = []
                    docx_params = []
                    for name in name_tensors.keys():
    #                     if ('topic_coords' in name) and (name not in prev_name_tensors.keys()):
    #                         non_init_coords.append(name)
                        if ('_docx' in name):
                            docx_params.append(name)
    #                 sess.run([name_tensors[name].assign(variable) for name, variable in prev_name_tensors.items()]) # restore parameters
                    sess.run(tf.global_variables_initializer())
#                     sess.run([name_tensors[name].assign(variable) for name, variable in prev_name_tensors.items()  if name in name_tensors.keys() and (name not in docx_params)])#and (name not in docx_params)
                    sess.run([name_tensors[name].assign(variable) for name, variable in prev_name_tensors.items()  if name in name_tensors.keys()])#and (name not in docx_params)

                    max_coord = sess.run(tf.reduce_max(tf.math.abs(tf.convert_to_tensor(list(model.topic_coords.values())))))

                    topic_coords = sess.run(model.topic_coords)

                    highest_p_dict = {}

                    for name in name_tensors.keys():
                        if ('Adagrad' not in name) and ('topic_coords' in name) and (name not in prev_name_tensors.keys()):
                            node_idx = int(name.split('topic_coords')[1].split(':')[0])
                            node_level = model.node_level[node_idx]

                            parent = model.child_to_parent_idxs[node_idx]
                            highest_p = 0
                            node_highest_p = parent

                            node_x = topic_coords.get(node_highest_p)[:,0][0]#/100.
                            node_y = topic_coords.get(node_highest_p)[:,1][0]#/100.

                            new_arr = np.array([[node_x+random.uniform(0, 0.01), node_y + random.uniform(0, 0.01)]], dtype=np.float32)
                            sess.run([name_tensors[name].assign(new_arr)])


                    saver = tf.train.Saver(max_to_keep=1)
                    gc.collect()
            time_start = time.time()

        train_batches = get_batches(instances_train, config.batch_size, iterator=True)
        epoch += 1



{0: [1, 2, 3], 1: [21, 22, 23], 2: [41, 42, 43], 3: [61, 62, 63]}
[0, 1, 2, 3, 21, 22, 23, 41, 42, 43, 61, 62, 63]
self.child_to_parent_idxs
{1: 0, 2: 0, 3: 0, 21: 1, 22: 1, 23: 1, 41: 2, 42: 2, 43: 2, 61: 3, 62: 3, 63: 3}
self.tree_depth:  {0: 1, 1: 2, 21: 3, 22: 3, 23: 3, 2: 2, 41: 3, 42: 3, 43: 3, 3: 2, 61: 3, 62: 3, 63: 3}
self.n_depth:  3
self.level_nodes:  {0: [0], 1: [1, 2, 3], 2: [21, 22, 23, 41, 42, 43, 61, 62, 63]}
child_to_parent_idxs get_leaf_parents {1: 0, 2: 0, 3: 0, 21: 1, 22: 1, 23: 1, 41: 2, 42: 2, 43: 2, 61: 3, 62: 3, 63: 3}
tree_idxs get_leaf_parents {0: [1, 2, 3], 1: [21, 22, 23], 2: [41, 42, 43], 3: [61, 62, 63]}








leaf_ancestor_idxs:  {21: [0, 1, 21], 22: [0, 1, 22], 23: [0, 1, 23], 41: [0, 2, 41], 42: [0, 2, 42], 43: [0, 2, 43], 61: [0, 3, 61], 62: [0, 3, 62], 63: [0, 3, 63]}
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-su

## VIS

* Tree structure: model.tree_idxs
* Node level: model.level_nodes
* Topic coordinates: model.topic_coords
* Doc coordinates: model.latents_bow